In [1]:
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from kdutils.common import fetch_temp_data, fetch_temp_returns

In [3]:
method = 'cicso0'
instruments = 'ims'
task_id = '200037'
period = 15
shift = 1

In [4]:
total_data = fetch_temp_data(
        method=method,
        instruments=instruments,
        task_id=task_id,
        datasets=['train'])
total_data.head()

./records/cicso0/ims/level2/train_data.feather


,trade_time,symbol,open,high,low,close,volume,money,twap,pct_change,...,price_imbalance_3,price_imbalance_4,depth_imbalance_0,depth_imbalance_1,depth_imbalance_2,depth_imbalance_3,depth_imbalance_4,ask_bid_press,realized_volatility,code
0,2022-07-25 09:31:00,IM2208,6971.8,6994.8,6964.4,6973.8,602,840545880.0,6981.585000,0.000287,...,-0.000548,-0.000727,0.048350,0.032361,0.080896,0.116902,-0.009520,0.191198,0.002546,IM
1,2022-07-25 09:32:00,IM2208,6973.8,6995.2,6969.6,6982.8,428,597671840.0,6982.126050,0.001291,...,-0.000485,-0.000618,0.077734,0.073275,0.029830,0.017027,0.020993,0.243969,0.001839,IM
2,2022-07-25 09:33:00,IM2208,6982.8,6989.8,6981.6,6989.8,280,391195160.0,6985.326496,0.001002,...,-0.000393,-0.000494,0.072032,0.123311,0.126821,0.172182,0.129718,0.337424,0.001727,IM
3,2022-07-25 09:34:00,IM2208,6989.6,6993.0,6982.6,6993.0,319,445873560.0,6988.716949,0.000486,...,-0.000357,-0.000442,-0.014664,-0.053867,0.042050,-0.065601,-0.069451,-0.099565,0.001665,IM
4,2022-07-25 09:35:00,IM2208,6993.6,7009.0,6991.8,7002.2,284,397659840.0,6999.337288,0.001230,...,-0.000445,-0.000558,0.056584,0.106953,0.097129,0.101817,0.024232,0.132744,0.001700,IM


In [5]:
market_data_indexed = total_data.set_index(['trade_time','code'])['close'].unstack()
market_data_indexed.head()

code,IM
trade_time,
2022-07-25 09:31:00,6973.8
2022-07-25 09:32:00,6982.8
2022-07-25 09:33:00,6989.8
2022-07-25 09:34:00,6993.0
2022-07-25 09:35:00,7002.2


In [6]:
#return1_data = ((market_data_indexed.shift(-period-shift) / market_data_indexed.shift(-shift)) - 1)
return1_data = np.log((market_data_indexed.shift(-period-shift) / market_data_indexed.shift(-shift)))
return1_data = return1_data.stack()
return1_data.head()

trade_time           code
2022-07-25 09:31:00  IM      0.000773
2022-07-25 09:32:00  IM     -0.001088
2022-07-25 09:33:00  IM     -0.001145
2022-07-25 09:34:00  IM     -0.003835
2022-07-25 09:35:00  IM     -0.006270
dtype: float64

In [8]:
pricep = market_data_indexed ##pricep 是每分钟的价格数据
pre_pricep = pricep.shift(1) ## 向下移动1位 # pre_pricep 是上一分钟的价格 P(t-1)
ret_v2v = np.log((pricep) / pre_pricep) ## t-1期 close ~ t期 close 的log收益率 ret_v2v 在时间 t 的值是 log( P(t) / P(t-1) )
yields_data = ret_v2v.shift(-2) ## 向上移动2位， t期 即 log( P(t+2) / P(t+1) ) 的值
yields_data = yields_data.stack()
yields_data.name = 'chg_pct'
yields_data = yields_data.reset_index()
df = yields_data.copy()
df.set_index("trade_time", inplace=True)
df['nxt1_ret'] = df['chg_pct']

df = df.groupby("code").rolling(window=period, min_periods=1)['nxt1_ret'].sum().groupby(level=0)
df = df.shift(0).unstack().T.shift(-(period + 0 - 1)).stack(dropna=False)
df.name = 'nxt1_ret'
return2_data = df
return2_data.head()

trade_time           code
2022-07-25 09:31:00  IM      0.000773
2022-07-25 09:32:00  IM     -0.001088
2022-07-25 09:33:00  IM     -0.001145
2022-07-25 09:34:00  IM     -0.003835
2022-07-25 09:35:00  IM     -0.006270
Name: nxt1_ret, dtype: float64

In [9]:
return1_data.corr(return2_data)

0.9999999999999999

In [10]:
returns_data = fetch_temp_returns(
        method=method,
        instruments=instruments,
        datasets=['train'],
        category='returns')
return4_data = returns_data.set_index(['trade_time','code'])['nxt1_ret_{0}h'.format(period)]
return4_data#.head()

./records/cicso0/ims/returns/train_returns1.feather


trade_time           code
2022-07-25 09:30:00  IM      0.003121
2022-07-25 09:31:00  IM      0.000773
2022-07-25 09:32:00  IM     -0.001088
2022-07-25 09:33:00  IM     -0.001145
2022-07-25 09:34:00  IM     -0.003835
                               ...   
2024-05-28 14:33:00  IM     -0.000265
2024-05-28 14:34:00  IM      0.001212
2024-05-28 14:35:00  IM      0.000265
2024-05-28 14:36:00  IM      0.000114
2024-05-28 14:37:00  IM     -0.000114
Name: nxt1_ret_15h, Length: 107461, dtype: float64

In [11]:
return4_data.corr(return2_data)

0.9616298979244734

In [12]:
indexs = return4_data.index.intersection(return2_data.index)
return2_data1 = return2_data.reindex(indexs)
return4_data1 = return4_data.reindex(indexs)

return4_data1.corr(return2_data1)

0.9616298979244734

In [13]:
return2_data1.loc['2022-11-07 15:00:00']

code
IM   -0.005451
Name: nxt1_ret, dtype: float64

In [14]:
return4_data1.loc['2022-11-07 15:00:00']

code
IM    0.097332
Name: nxt1_ret_15h, dtype: float64

In [15]:
rr1 = (return2_data1 - return4_data1)
rr1.name = 'value'
rr1 = rr1.reset_index()
rr1['abs_value'] = rr1['value'].abs()
rr1.sort_values(by=['abs_value'])

,trade_time,code,value,abs_value
5884,2022-08-26 13:05:00,IM,0.000000,0.000000
17577,2022-11-11 10:28:00,IM,0.000000,0.000000
17578,2022-11-11 10:29:00,IM,0.000000,0.000000
17579,2022-11-11 10:30:00,IM,0.000000,0.000000
17580,2022-11-11 10:31:00,IM,0.000000,0.000000
...,...,...,...,...
9358,2022-09-16 14:59:00,IM,-0.023646,0.023646
9354,2022-09-16 14:55:00,IM,-0.024842,0.024842
9345,2022-09-16 14:46:00,IM,-0.025283,0.025283
16784,2022-11-07 14:45:00,IM,0.102213,0.102213


In [16]:
rr1[rr1['abs_value'] > 10e-03].sort_values(by=['abs_value'])#.tail(40)

,trade_time,code,value,abs_value
90712,2024-02-07 14:55:00,IM,-0.010045,0.010045
95512,2024-03-14 14:55:00,IM,0.010070,0.010070
9116,2022-09-15 14:57:00,IM,0.010166,0.010166
95504,2024-03-14 14:47:00,IM,0.010196,0.010196
95515,2024-03-14 14:58:00,IM,0.010201,0.010201
...,...,...,...,...
9358,2022-09-16 14:59:00,IM,-0.023646,0.023646
9354,2022-09-16 14:55:00,IM,-0.024842,0.024842
9345,2022-09-16 14:46:00,IM,-0.025283,0.025283
16784,2022-11-07 14:45:00,IM,0.102213,0.102213
